In [1]:
# you could do with some help from the given types and functions ;)
# below are some usage examples
from polygon_utils import *

# Carlton Gardens polygon
carltonGardensPerimeterLoop = [
    (144.970087, -37.800805), (144.974108, -37.801204), (144.973073, -37.807611), (144.968972, -37.807191)
]
carltonGardensLakeLoop = [
    (144.969768, -37.805891), (144.970074, -37.806206), (144.969954, -37.806332), (144.969583, -37.806153),
    (144.969689, -37.805985)
]
carltonGardensPolygon = [carltonGardensPerimeterLoop, carltonGardensLakeLoop]

# Royal Exhibition Building polygon
exhibitionBuildingPolygon = [
    [
        (144.972502, -37.804601), (144.972396, -37.805126), (144.970485, -37.804884), (144.970565, -37.804350),
        (144.971369, -37.804443), (144.971420, -37.804191), (144.971804, -37.804228), (144.971760, -37.804487)
    ]
]

# WeWork block polygon
weWorkBlockPolygon = [
    [
        (144.963286, -37.814212), (144.964498, -37.813854), (144.964962, -37.814806), (144.963711, -37.815168)
    ]
]

carltonGardensArea = multi_polygon_area([carltonGardensPolygon])
print(carltonGardensArea)
exhibitionBuildingArea = multi_polygon_area([exhibitionBuildingPolygon])
print(exhibitionBuildingArea)
weWorkBlockArea = multi_polygon_area([weWorkBlockPolygon])
print(weWorkBlockArea)

merge = merge_multi_polygons([exhibitionBuildingPolygon], [weWorkBlockPolygon])
def approximately(a, b):
    return abs(a - b) < 0.000000001
assert approximately((exhibitionBuildingArea + weWorkBlockArea), multi_polygon_area(merge))

assert may_intersect([carltonGardensPolygon], [exhibitionBuildingPolygon])
assert intersection_area([carltonGardensPolygon], [exhibitionBuildingPolygon]) > 0.0

assert not may_intersect([carltonGardensPolygon], [weWorkBlockPolygon])
assert intersection_area([carltonGardensPolygon], [weWorkBlockPolygon]) == 0.0

2.6311408499972285e-05
1.159406999999211e-06
1.3348710000033568e-06


In [1]:
!pip install Shapely==1.6.4.post2
!pip install pyspark==2.3.1

sagemaker-pyspark 1.2.6 has requirement pyspark==2.3.2, but you'll have pyspark 2.3.1 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
sagemaker-pyspark 1.2.6 has requirement pyspark==2.3.2, but you'll have pyspark 2.3.1 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [115]:
import json
import csv
import re
from pyspark import SparkContext
from pyspark.sql import SQLContext, functions
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, StringType, BooleanType, ArrayType
from polygon_utils import *
from shapely.geometry import shape, Point, mapping
from shapely.wkt import loads, dumps

In [121]:
# Roll-up polygons to suburban boundaries
suburbs = {}
with open('./melb_inner_2016.json') as json_file:
    for line in json_file:
        feat = json.loads(line)
        suburb = feat['sa2_name16']
        new_geom = shape(feat['geometry'])    
        if not suburbs or suburb not in suburbs:
            suburbs[suburb] = new_geom
        else:
            suburbs[suburb] = unary_union([suburbs[suburb], new_geom])

melb_suburb_2016 = []
for suburb, geom in suburbs.items():
    #melb_suburb_2016.append({'suburb_name': suburb, 'poly': mapping(geom)['coordinates']})
    melb_suburb_2016.append({'suburb_name': suburb, 'poly': dumps(geom)})

In [122]:
with open('./melb_suburb_2016.json', 'w') as suburb_json:
    json.dump(melb_suburb_2016, suburb_json)

In [ ]:
sc = SparkContext()
sc.addPyFile('polygon_utils.py')
sqlc = SQLContext(sc)

In [142]:
# Load suburban polygons
sa2 = sqlc.read.json('./melb_suburb_2016.json')

In [87]:
# Load urban forests
csv_schema = StructType([
    StructField('area', FloatType(), nullable = False),
    StructField('the_geom', StringType(), nullable = False)
])
urban_forest = sqlc.read.csv('./melb_urban_forest_2016.txt/part-0000[0-5]', schema = csv_schema, sep=' ')

In [133]:
# Fix up the_geom as proper WKT
#insert_comma = re.compile('\s([^\s]*)\s([^\s]*)')
insert_comma = re.compile('(\s[^\s]*)\s')
def insert_commas(the_geom):
    gtype, lonlat = the_geom.split(' ', 1)
    gtype = gtype.strip()
    #lonlat = insert_comma.sub(r' (\1,\2)', ' ' + lonlat.strip()).strip()
    lonlat = insert_comma.sub(r'\1, ', lonlat.strip())
    #return f'{lonlat}'
    return f'{gtype} {lonlat}'

urban_forest_fixed = urban_forest.withColumn('new_geom', functions.udf(insert_commas, StringType())('the_geom'))\
.drop('the_geom').withColumnRenamed('new_geom', 'the_geom').drop('area')

In [145]:
def get_rect(poly):
    return loads(poly).bounds

get_rect_udf = functions.udf(get_rect, ArrayType(FloatType()))

urban_forest_fixed = urban_forest_fixed.withColumn('urban_forest_rect', get_rect_udf('the_geom'))
sa2 = sa2.withColumn('sa2_rect', get_rect_udf('poly'))

In [172]:
def get_area(poly):
    return loads(poly).area

get_area_udf = functions.udf(get_area, FloatType())

sa2 = sa2.withColumn('sa2_area', get_area_udf('poly'))

In [175]:
def may_intersect(rect1, rect2):
    min_lon_1, min_lat_1, max_lon_1, max_lat_1 = rect1
    min_lon_2, min_lat_2, max_lon_2, max_lat_2 = rect2
    return min_lat_1 <= max_lat_2 and \
           max_lon_1 >= min_lon_2 and \
           max_lat_1 >= min_lat_2 and \
           min_lon_1 <= max_lon_2

sa2_p1 = sa2.repartition(1)
urban_forest_p10 = urban_forest_fixed.repartition(10)
urban_forest_in_sa2 = sa2_p1.crossJoin(urban_forest_p10).where(functions.udf(may_intersect, BooleanType())\
                                                  (sa2_p1.sa2_rect, urban_forest_p10.urban_forest_rect))

In [176]:
urban_forest_in_sa2 = urban_forest_in_sa2.drop('sa2_rect').drop('urban_forest_rect')

In [197]:
def get_veg_rate(sa2_geom, urban_forest_geom, sa2_area):
    return (loads(sa2_geom).intersection(loads(urban_forest_geom))).area / sa2_area

suburban_vegetation_rate = urban_forest_in_sa2.withColumn('urban_forest_%_contribution_to_sa2',\
                                                     functions.udf(get_veg_rate, FloatType())\
                                                     ('poly', 'the_geom', 'sa2_area'))\
.drop('poly').drop('the_geom').drop('sa2_area')

In [ ]:
suburban_vegetation_rate.head()

In [198]:
answer = suburban_vegetation_rate.groupBy('suburb_name').sum('urban_forest_%_contribution_to_sa2')
answer.orderBy('urban_forest_%_contribution_to_sa2', ascending=False).show()

AnalysisException: "cannot resolve '`urban_forest_%_contribution_to_sa2`' given input columns: [suburb_name, sum(urban_forest_%_contribution_to_sa2)];;\n'Sort ['urban_forest_%_contribution_to_sa2 DESC NULLS LAST], true\n+- AnalysisBarrier\n      +- Aggregate [suburb_name#378], [suburb_name#378, sum(cast(urban_forest_%_contribution_to_sa2#706 as double)) AS sum(urban_forest_%_contribution_to_sa2)#729]\n         +- Project [suburb_name#378, urban_forest_contribution_to_sa2#646, urban_forest_%_contribution_to_sa2#706]\n            +- Project [suburb_name#378, sa2_area#501, urban_forest_contribution_to_sa2#646, urban_forest_%_contribution_to_sa2#706]\n               +- Project [suburb_name#378, sa2_area#501, the_geom#334, urban_forest_contribution_to_sa2#646, urban_forest_%_contribution_to_sa2#706]\n                  +- Project [poly#377, suburb_name#378, sa2_area#501, the_geom#334, urban_forest_contribution_to_sa2#646, get_veg_rate(poly#377, the_geom#334, sa2_area#501) AS urban_forest_%_c